<a href="https://colab.research.google.com/github/MosheWasserb/PyTorchNotbooks/blob/master/STS2_BERT_FT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 44.2MB/s 
     |████████████████████████████████| 1.0MB 39.2MB/s 
     |████████████████████████████████| 890kB 46.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=85cb57333aacd86bf8b5970e592ab860620429f3ba6d1850c7952cc403e57e5b
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from sklearn import preprocessing
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.preprocessing import LabelEncoder
from IPython.display import clear_output
from torch.nn.utils import clip_grad_norm_
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier

### Helper functions
import pickle

def convert_to_pickle(item, directory):
    pickle.dump(item, open(directory,"wb"))


def load_from_pickle(directory):
    return pickle.load(open(directory,"rb"))

Importing the dataset

In [3]:
# Import Training, Validation and Test csvs
data = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

text_col=data.columns.values[0] 
category_col=data.columns.values[1]

data.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


Init and set data

In [0]:
TEST_SIZE=0.2
RANDOM_STATE=42
SAMPLE_SIZE = 2000
UNLABEL_SIZE = 4000 

max_bert_len = 70

BATCH_SIZE=64
EPOCHS =5
HIDDEN_SIZE=768

catagories=list(set(data[category_col].unique()))
OUTPUT_DIM= len(catagories)   #num of catagories

In [0]:
batch_1 = data.sample(n=SAMPLE_SIZE, random_state=RANDOM_STATE);
residue_batch = data.drop(batch_1.index)
unlabel_texts_batch = residue_batch.sample(n=UNLABEL_SIZE, random_state=RANDOM_STATE);
unlabel_texts=unlabel_texts_batch[text_col]

In [0]:
train_texts, test_texts, train_labels, test_labels = train_test_split(batch_1[text_col],batch_1[category_col], test_size=TEST_SIZE,random_state=RANDOM_STATE)

In [96]:
len(train_texts), len(test_texts), len(train_labels), len(test_labels)

(1600, 400, 1600, 400)

1. Ngram baseline model

In [97]:
#CounterVecorizer(ngram_range=(1,3), min_df=0.2, max_df=0.7, max_features=10000, stop_words="english")
#ngramCount_baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,2)), LogisticRegression()).fit(train_texts, train_labels)
#ngramCount_baseline_model = make_pipeline(TfidfVectorizer(ngram_range=(1,2)), LogisticRegression()).fit(train_texts, train_labels)
ngramCount_baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,2),stop_words="english",max_features=30000, max_df=0.75), LogisticRegression()).fit(train_texts, train_labels)
ngramCount_baseline_predicted = ngramCount_baseline_model.predict(test_texts)
print(classification_report(test_labels, ngramCount_baseline_predicted))
print(confusion_matrix(test_labels, ngramCount_baseline_predicted))

              precision    recall  f1-score   support

           0       0.73      0.67      0.70       202
           1       0.69      0.75      0.72       198

    accuracy                           0.71       400
   macro avg       0.71      0.71      0.71       400
weighted avg       0.71      0.71      0.71       400

[[135  67]
 [ 49 149]]


In [286]:
ngramCount_baseline_model.score(test_texts,test_labels)

0.5033333333333333

1.1 NGRAM SGDClassifier 

In [336]:
ngramCount_baseline_model = make_pipeline(TfidfVectorizer(ngram_range=(1,2)), SGDClassifier(loss = 'log')).fit(train_texts, train_labels)
ngramCount_baseline_predicted = ngramCount_baseline_model.predict(test_texts)
print(classification_report(test_labels, ngramCount_baseline_predicted))
print(confusion_matrix(test_labels, ngramCount_baseline_predicted))

              precision    recall  f1-score   support

           0       0.81      0.64      0.71       202
           1       0.70      0.84      0.76       198

    accuracy                           0.74       400
   macro avg       0.75      0.74      0.74       400
weighted avg       0.75      0.74      0.74       400

[[129  73]
 [ 31 167]]


2. Glove mean Logistic

In [0]:
### read data from your Google Drive
from google.colab import drive
drive.mount('/gdrive')

In [117]:
# define dict to hold a word and its vector
glove = {}
# read the word embeddings file ~820MB
f = open('/gdrive/My Drive/Colab Notebooks/DAIR/EmotionDataSet/glove.6B.100d.txt', encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove[word] = coefs
f.close()
# check the length
len(glove) # 400000

400000

In [0]:
def MeanSentenceGlove(texts):
# create vector for each sentences
# list to hold vector 
  sentence_vectors = []
# create vector for each clean normalized sentence
  for i in texts:
      if len(i) != 0:
          v = sum([glove.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
          #print(len(i))
      else:
          v = np.zeros((100,))
      sentence_vectors.append(v)
  #print('Total vectors created:',len(sentence_vectors))
  return(sentence_vectors)

In [119]:
train_glove = MeanSentenceGlove(train_texts)
test_glove = MeanSentenceGlove(test_texts)
Glove_baseline_model=LogisticRegression()
Glove_baseline_model.fit(train_glove, train_labels)
Glove_baseline_predicted = Glove_baseline_model.predict(test_glove)
print(classification_report(test_labels, Glove_baseline_predicted))

              precision    recall  f1-score   support

           0       0.76      0.73      0.75       202
           1       0.74      0.76      0.75       198

    accuracy                           0.75       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.75      0.75      0.75       400



In [0]:
temp = MeanSentenceGlove(train_texts)

Total vectors created: 64


In [0]:
temp[0]

2.1 Glove - NLTK tokenize

In [0]:
 #We'll use Average Glove here 
from nltk import word_tokenize
import nltk
nltk.download('punkt')

In [0]:
def AvgGlove(texts):
# create vector for each sentences
# list to hold vector 
  vectors = []
# create vector for each clean normalized sentence
  for i in texts:
      #vectors.append(np.average(glove.get(word_tokenize(i)), axis =0))
      vectors.append(np.average([glove.get(w, np.zeros((100,))) for w in word_tokenize(i)], axis = 0))
  return np.array(vectors)

In [299]:
train_glove = AvgGlove(train_texts)
test_glove = AvgGlove(test_texts)
Glove_baseline_model=LogisticRegression()
Glove_baseline_model.fit(train_glove, train_labels)
Glove_baseline_predicted = Glove_baseline_model.predict(test_glove)
print(classification_report(test_labels, Glove_baseline_predicted))

              precision    recall  f1-score   support

           0       0.59      0.86      0.70       426
           1       0.78      0.47      0.59       474

    accuracy                           0.65       900
   macro avg       0.69      0.66      0.64       900
weighted avg       0.69      0.65      0.64       900



2.2 Glove Weighted average tfidf

In [300]:
tfidf = TfidfVectorizer()
tfidf.fit(train_texts)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
# Now lets create a dict so that for every word in the corpus we have a corresponding IDF value
idf_dict = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [0]:
# Same as Avg Glove except instead of doing a regular average, we'll use the IDF values as weights.

def tfidfGlove(texts):
# create vector for each sentences
# list to hold vector 
  vectors = []
# create vector for each clean normalized sentence
  for i in texts:
      weights = [idf_dict.get(word, 1) for word in word_tokenize(i)]
      vectors.append(np.average([glove.get(w, np.zeros((100,))) for w in word_tokenize(i)], axis = 0, weights = weights))
  return np.array(vectors)

In [303]:
train_glove = tfidfGlove(train_texts)
test_glove = tfidfGlove(test_texts)
Glove_baseline_model=LogisticRegression()
Glove_baseline_model.fit(train_glove, train_labels)
Glove_baseline_predicted = Glove_baseline_model.predict(test_glove)
print(classification_report(test_labels, Glove_baseline_predicted))

              precision    recall  f1-score   support

           0       0.58      0.86      0.69       426
           1       0.78      0.44      0.57       474

    accuracy                           0.64       900
   macro avg       0.68      0.65      0.63       900
weighted avg       0.69      0.64      0.63       900



2.3 Glove Weighted average Count

In [304]:
count = CountVectorizer()
count.fit(train_texts)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
# Now lets create a dict so that for every word in the corpus we have a corresponding IDF value
count_dict = count.vocabulary_

In [0]:
# Same as Avg Glove except instead of doing a regular average, we'll use the IDF values as weights.

def countGlove(texts):
# create vector for each sentences
# list to hold vector 
  vectors = []
# create vector for each clean normalized sentence
  for i in texts:
      weights = [count_dict.get(word, 1) for word in word_tokenize(i)]
      vectors.append(np.average([glove.get(w, np.zeros((100,))) for w in word_tokenize(i)], axis = 0, weights = weights))
  return np.array(vectors)

In [307]:
train_glove = countGlove(train_texts)
test_glove = countGlove(test_texts)
Glove_baseline_model=LogisticRegression()
Glove_baseline_model.fit(train_glove, train_labels)
Glove_baseline_predicted = Glove_baseline_model.predict(test_glove)
print(classification_report(test_labels, Glove_baseline_predicted))

              precision    recall  f1-score   support

           0       0.51      0.91      0.65       426
           1       0.72      0.22      0.33       474

    accuracy                           0.54       900
   macro avg       0.62      0.56      0.49       900
weighted avg       0.62      0.54      0.48       900



3. FastText

In [0]:
from gensim.models import FastText

In [0]:
from gensim.utils import tokenize

In [174]:
list(tokenize('I like Moshe food', lowercase=True))

['i', 'like', 'moshe', 'food']

In [0]:
sentences_tokens=[[token for token in tokenize(sentence)] for sentence in train_texts]

In [0]:
model_fasttext_sts2 = FastText(sentences_tokens, size=100, window=5, min_count=0, workers=4,sg=0)

In [0]:
model_fasttext_sts2.most_similar('eat')


4. BERT Fine-tune

Load BERT pre-trained model

In [0]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

## Want Roberta instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.RobertaModel, ppb.RobertaTokenizer, 'roberta-base')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
BertModel = model_class.from_pretrained(pretrained_weights)

Tokenization

In [0]:
import logging
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)
tokenized = batch_1[text_col].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Limit tokenized to 512 max_bert_len

In [0]:
#padded = np.array([if (len(i)>0) i=3, for i in tokenized.values])
tokenized_limted = []
for i in tokenized.values:
  if len(i)>max_bert_len:
    tokenized_limted.append(i[:max_bert_len])
  else:
    tokenized_limted.append(i)
     


Padding 

In [0]:
padded = np.array([i + [0]*(max_bert_len-len(i)) for i in tokenized_limted])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [102]:
np.array(padded).shape

(2000, 70)

Masking

In [103]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(2000, 70)

Convert to tensors

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

**And now Deep Learning**

Biniarizatiom

In [0]:
mlb = preprocessing.MultiLabelBinarizer()
data_labels =  [set(catos) & set(catagories) for catos in batch_1[[category_col]].values]
bin_catagories = mlb.fit_transform(data_labels)
target_tensor_bin = torch.tensor(bin_catagories.tolist())

target_tensor=torch.max(target_tensor_bin,1)[1]

Convert to tensor

In [0]:
train_tokens_tensor, test_tokens_tensor, train_masks_tensor, test_masks_tensor = train_test_split(input_ids,attention_mask,test_size=TEST_SIZE,random_state=RANDOM_STATE)
train_y_tensor, test_y_tensor  = train_test_split(target_tensor,test_size=TEST_SIZE,random_state=RANDOM_STATE)

Dataloader 

In [0]:
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

Define model

In [0]:
class BertMultiClassifier(torch.nn.Module):
    def __init__(self, dropout=0.1, freeze_bert = False):
        super(BertMultiClassifier, self).__init__()
        # Need to define the right layer 
        #self.bert = model_class.from_pretrained(pretrained_weights)
        self.bert = BertModel.from_pretrained(pretrained_weights)
        #self.dropout = torch.nn.Dropout(dropout)

        if freeze_bert:
            for p in self.bert.parameters():
                p.requires_grad = False

        self.linear = torch.nn.Linear(HIDDEN_SIZE, OUTPUT_DIM)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask):
        last_hidden_states = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = last_hidden_states[0][:,0,:]
        #dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(pooled_output)
        proba = self.sigmoid(linear_output)
        return linear_output

In [0]:
bert_clf = BertMultiClassifier(freeze_bert=False)
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss() # the same as log_softmax + NLLLoss.  Check BCEWithLogitsLoss()


In [114]:
losses = []
steps = []
step = 0
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        probas = bert_clf(token_ids, masks)
        
        batch_loss = criterion(probas, labels)
        train_loss += batch_loss.item()
        
        
        bert_clf.zero_grad()
        batch_loss.backward()
        

        #clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("{0}/{1} loss: {2} ".format(step_num, len(train_y_tensor) / BATCH_SIZE, train_loss / (step_num + 1)))
        losses.append(batch_loss.item())
        steps.append(step)
        step += 1

#convert_to_pickle(bert_clf, "/gdrive/My Drive/Colab Notebooks/STS2/DistillBERT.pkl")
# Other option to save: torch.save(net.state_dict(), os.path.join(config["outputFolder"], config["outputFileName"]))


Epoch:  1
0/25.0 loss: 0.6954367160797119 


KeyboardInterrupt: ignored

In [0]:
#convert_to_pickle(bert_clf, "/gdrive/My Drive/Colab Notebooks/STS2/DistillBERT.pkl")

Evaluation

In [0]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t for t in batch_data)

        probas = bert_clf(token_ids, masks)
        
        bert_predicted += list(torch.max(probas,1)[1])

In [53]:
print(classification_report(test_y_tensor, bert_predicted))
print(confusion_matrix(test_y_tensor, bert_predicted))

              precision    recall  f1-score   support

           0       0.91      0.88      0.90       202
           1       0.88      0.91      0.90       198

    accuracy                           0.90       400
   macro avg       0.90      0.90      0.90       400
weighted avg       0.90      0.90      0.90       400

[[178  24]
 [ 17 181]]


4. BERT freeze 

In [0]:
with torch.no_grad():
    last_hidden_states = BertModel(input_ids, attention_mask=attention_mask)
features = last_hidden_states[0][:,0,:].numpy()

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, batch_1[category_col],test_size=TEST_SIZE,random_state=RANDOM_STATE)

In [329]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [330]:
lr_clf.score(test_features, test_labels)

0.7744444444444445

5. 

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [275]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.492 (+/- 0.06)


To save time load BERT model and create logit for labeled and unlabeled data

In [12]:
### read data from your Google Drive
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# Read BERT model from pkl

class BertMultiClassifier(torch.nn.Module):
    def __init__(self, dropout=0.1):
        super(BertMultiClassifier, self).__init__()
        # Need to define the right layer 
        self.bert = model_class.from_pretrained(pretrained_weights)
        #self.dropout = torch.nn.Dropout(dropout)
        self.linear = torch.nn.Linear(HIDDEN_SIZE, OUTPUT_DIM)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask):
        last_hidden_states = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = last_hidden_states[0][:,0,:]
        #dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(pooled_output)
        proba = self.sigmoid(linear_output)
        return linear_output

bert_clf = load_from_pickle("/gdrive/My Drive/Colab Notebooks/STS2/DistillBERT.pkl")

6. Distill with training

In [0]:
# Create logits
train_logits = build_bert_logits(train_texts, bert_clf, tokenizer, batch_size=BATCH_SIZE)

In [0]:
bert_predicted=torch.max(torch.tensor(train_logits),1)[1]

In [129]:
test_logits[0:4]

array([[-2.2830608,  2.4730306],
       [-1.6793319,  1.7918842],
       [-2.3423913,  1.9774685],
       [-2.579123 ,  2.825541 ]], dtype=float32)

In [131]:
distilled_predicted_logits[0:4]

array([[-0.9564814 ,  0.87353061],
       [-2.0427931 ,  2.21993311],
       [-4.04284265,  4.23176752],
       [ 0.24665368, -0.05799119]])

In [121]:
print(classification_report(train_y_tensor, bert_predicted))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       759
           1       1.00      1.00      1.00       841

    accuracy                           1.00      1600
   macro avg       1.00      1.00      1.00      1600
weighted avg       1.00      1.00      1.00      1600



In [0]:
# Run regression 
distilled_model = make_pipeline(CountVectorizer(ngram_range=(1,2)), LinearRegression()).fit(train_texts, train_logits)
distilled_predicted_logits = distilled_model.predict(test_texts)
distilled_bert_predicted=torch.max(torch.tensor(distilled_predicted_logits),1)[1]
print(classification_report(test_y_tensor, distilled_bert_predicted))


7. Distill with unlabeled

In [0]:
# Create logits
unlabel_logits = build_bert_logits(unlabel_texts, bert_clf, tokenizer, batch_size=BATCH_SIZE)

In [76]:
# Run regression
unlabel_model = make_pipeline(CountVectorizer(ngram_range=(1,2)), LinearRegression()).fit(unlabel_texts, unlabel_logits)
unlabel_predicted_logits = unlabel_model.predict(test_texts)
unlabel_bert_predicted=torch.max(torch.tensor(unlabel_predicted_logits),1)[1]
print(classification_report(test_y_tensor,unlabel_bert_predicted))

              precision    recall  f1-score   support

           0       0.78      0.68      0.73       202
           1       0.71      0.81      0.76       198

    accuracy                           0.74       400
   macro avg       0.75      0.75      0.74       400
weighted avg       0.75      0.74      0.74       400



8. Distill with unlabel with Glove

Init/Build train_y_tensor and test_y_tensor

In [0]:
catagories=list(set(data[category_col].unique()))

mlb = preprocessing.MultiLabelBinarizer()
data_labels =  [set(catos) & set(catagories) for catos in batch_1[[category_col]].values]
bin_catagories = mlb.fit_transform(data_labels)
target_tensor_bin = torch.tensor(bin_catagories.tolist())

target_tensor=torch.max(target_tensor_bin,1)[1]
train_y_tensor, test_y_tensor  = train_test_split(target_tensor,test_size=TEST_SIZE,random_state=RANDOM_SEED)

In [0]:
print(vec_matrix)

<bound method _cs_matrix.toarray of <6400x149035 sparse matrix of type '<class 'numpy.int64'>'
	with 301263 stored elements in Compressed Sparse Row format>>


Simple regression - N-gram count only (baseline)

In [0]:
ngramCount_baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LogisticRegression()).fit(train_texts, train_labels)
ngramCount_baseline_predicted = ngramCount_baseline_model.predict(test_texts)
print(classification_report(test_labels, ngramCount_baseline_predicted))

              precision    recall  f1-score   support

       anger       0.90      0.64      0.75       229
        fear       0.80      0.61      0.69       175
         joy       0.72      0.92      0.81       517
        love       0.83      0.38      0.52       129
     sadness       0.78      0.89      0.83       485
    surprise       0.83      0.29      0.43        65

    accuracy                           0.77      1600
   macro avg       0.81      0.62      0.67      1600
weighted avg       0.79      0.77      0.76      1600



BERT Embedding (freeze BERT)

BERT Finetuning 

Distill with training

In [0]:
#from sklearn import linear_model
#reg = linear_model.Ridge(alpha=.5)
#reg = linear_model.Lasso(alpha=.1)

In [0]:
distilled_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LinearRegression()).fit(train_texts, train_logits_load)
distilled_predicted_logits = distilled_model.predict(test_texts)
distilled_bert_predicted=torch.max(torch.tensor(distilled_predicted_logits),1)[1]
print(classification_report(test_y_tensor, distilled_bert_predicted))

              precision    recall  f1-score   support

           0       0.93      0.80      0.86       229
           1       0.81      0.79      0.80       175
           2       0.79      0.94      0.86       517
           3       0.85      0.52      0.64       129
           4       0.88      0.91      0.90       485
           5       0.87      0.52      0.65        65

    accuracy                           0.84      1600
   macro avg       0.86      0.75      0.79      1600
weighted avg       0.85      0.84      0.84      1600



Distill with unlabeled samples 

In [0]:
unlabeled_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LinearRegression()).fit(unlabeled_texts, unlabeled_logits)
unlabeled_predicted_logits = unlabeled_model.predict(test_texts)
unlabeled_bert_predicted=torch.max(torch.tensor(unlabeled_predicted_logits),1)[1]
print(classification_report(test_y_tensor,unlabeled_bert_predicted))

              precision    recall  f1-score   support

           0       0.89      0.76      0.82       229
           1       0.77      0.77      0.77       175
           2       0.77      0.89      0.82       517
           3       0.71      0.50      0.58       129
           4       0.85      0.89      0.87       485
           5       0.94      0.49      0.65        65

    accuracy                           0.81      1600
   macro avg       0.82      0.72      0.75      1600
weighted avg       0.81      0.81      0.80      1600



Distill with unlabeled samples + training 

In [0]:
mix_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LinearRegression()).fit(np.concatenate((train_texts,unlabeled_texts),axis=0), np.concatenate((train_logits_load,unlabeled_logits.to_numpy())))
mix_predicted_logits = mix_model.predict(test_texts)
mix_bert_predicted=torch.max(torch.tensor(mix_predicted_logits),1)[1]
print(classification_report(test_y_tensor,mix_bert_predicted))

NameError: ignored

Distill with STS data

In [0]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [0]:
class BertMultiClassifier(torch.nn.Module):
    def __init__(self, dropout=0.1):
        super(BertMultiClassifier, self).__init__()
        # Need to define the right layer 
        self.bert = model_class.from_pretrained(pretrained_weights)
        #self.dropout = torch.nn.Dropout(dropout)
        self.linear = torch.nn.Linear(HIDDEN_SIZE, OUTPUT_DIM)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask):
        last_hidden_states = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = last_hidden_states[0][:,0,:]
        #dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(pooled_output)
        proba = self.sigmoid(linear_output)
        return linear_output
        
bert_clf = load_from_pickle("/gdrive/My Drive/Colab Notebooks/DAIR/EmotionDataSet/DistilBERTfineTune6400tr5epochs.pkl")

In [0]:
labeled_data = data.sample(n=TRAINING_PLUS_TEST_SIZE, random_state=RANDOM_SEED); 
train_texts, test_texts, train_labels, test_labels = train_test_split(labeled_data['text'], labeled_data['emotions'],test_size=TEST_SIZE,random_state=RANDOM_SEED)

In [0]:
catagories=list(set(data[category_col].unique()))

mlb = preprocessing.MultiLabelBinarizer()
data_labels =  [set(catos) & set(catagories) for catos in batch_1[[category_col]].values]
bin_catagories = mlb.fit_transform(data_labels)
target_tensor_bin = torch.tensor(bin_catagories.tolist())

target_tensor=torch.max(target_tensor_bin,1)[1]
train_y_tensor, test_y_tensor  = train_test_split(target_tensor,test_size=TEST_SIZE,random_state=RANDOM_SEED)

In [0]:
BATCH_SIZE=64
sts_data = df.sample(n=6400, random_state=42)
sts_unlabeled_texts = sts_data[0]
sts_logits=build_bert_logits(sts_unlabeled_texts,bert_clf, tokenizer, batch_size=BATCH_SIZE)

In [0]:
sts_unlabeled_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LinearRegression()).fit(sts_unlabeled_texts, sts_logits)
sts_unlabeled_predicted_logits = sts_unlabeled_model.predict(test_texts)
sts_unlabeled_bert_predicted=torch.max(torch.tensor(sts_unlabeled_predicted_logits),1)[1]
print(classification_report(test_y_tensor,sts_unlabeled_bert_predicted))

              precision    recall  f1-score   support

           0       0.10      0.00      0.01       229
           1       0.33      0.02      0.03       175
           2       0.39      0.86      0.54       517
           3       0.60      0.02      0.04       129
           4       0.48      0.42      0.45       485
           5       0.55      0.09      0.16        65

    accuracy                           0.41      1600
   macro avg       0.41      0.24      0.20      1600
weighted avg       0.39      0.41      0.32      1600



In [0]:
def build_bert_logits(texts, bert_clf,tokenizer, batch_size):

  tokenized = texts.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

  tokenized_limted = []
  for i in tokenized.values:
    if len(i)>max_bert_len:
      tokenized_limted.append(i[:max_bert_len])
    else:
      tokenized_limted.append(i)

  padded = np.array([i + [0]*(max_bert_len-len(i)) for i in tokenized_limted])

  attention_mask = np.where(padded != 0, 1, 0)

  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  #tokens_tensor = torch.tensor(input_ids)
  #masks_tensor = torch.tensor(attention_mask)

  dataset = TensorDataset(input_ids, attention_mask)
  dataloader = DataLoader(dataset, batch_size)

  bert_clf.eval()
  list_logits = []
  with torch.no_grad():
      for step_num, batch_data in enumerate(dataloader):

          token_ids, masks = tuple(t for t in batch_data)

          logits = bert_clf(token_ids, masks)
        
          list_logits += list(logits)

          print("{0}/{1}".format(step_num, len(dataset) / batch_size))

  list_logits_numpy= (i.numpy() for i in list_logits)
  logits = np.vstack(list_logits_numpy)
  return(logits)


**N-gram plus Glove embedding** 
Example code: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/3%20-%20Faster%20Sentiment%20Analysis.ipynb 

https://colab.research.google.com/github/MosheWasserb/PyTorchNotbooks/blob/master/Glove_CNN_MultiClass.ipynb#scrollTo=tnUazzVHSuB6

Simple logistic by replace CounteVecorize with mean sentence Glove 

Distill with Back translate

Distill with VAT